# Bewertung von Klassifikationsmodellen

## 1. Accuracy

- Wie viele der vorhersagen hat das Modell korrekt gemacht?

$$\text{Accuracy} = \frac{\text{Anzahl korrekter Vorhersagen}}{\text{Gesamtzahl der Vorhersagen}}=\frac{TP+TN}{TP+FP+FN+TN}$$
 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
heart_disease = pd.read_csv("data/heart-disease.csv")
X = heart_disease.drop("target", axis=1)
y = heart_disease["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [3]:
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.2%}")

Accuracy: 86.89%


In [4]:
acc = clf.score(X_test, y_test)
print(f"Accuracy: {acc:.2%}")

Accuracy: 86.89%


## Probleme der Accuracy

**Unausgeglichene (unbalanced) Datensätze:**



In [5]:
heart_disease["target"].value_counts()

target
1    165
0    138
Name: count, dtype: int64

In [6]:
heart_disease["target"].value_counts(normalize=True)

target
1    0.544554
0    0.455446
Name: proportion, dtype: float64

- 50 zu 50 ist perfekt
- 60 zu 40 oder 70 zu 30 ist in vielen Fällen okay
- 90 zu 10 sehr problematisch

In [7]:
import plotly.express as px

class_counts = heart_disease["target"].value_counts().reset_index()
class_counts.columns = ["Klasse", "Anzahl"]
class_counts

,Klasse,Anzahl
0,1,165
1,0,138


In [8]:
fig = px.bar(
    class_counts,
    x="Klasse",
    y="Anzahl",
    text="Anzahl",
    title="Verteilung der Zielklassen",
    labels={"Klasse": "Zielklasse", "Anzahl": "Anzahl der Werte"},
    color="Klasse"
)

fig

### Probleme der Accuracy
 
**Keine Unterscheidung zwischen Arten von Fehlern:**
 
| Vorhersage / Realität | Positiv (1)             | Negativ (0)             |
| --------------------- | ----------------------- | ----------------------- |
| **Positiv (1)**       | **True Positive** (TP)  | **False Positive** (FP) |
| **Negativ (0)**       | **False Negative** (FN) | **True Negative** (TN)  |
 
Lösung: `confusion_matrix()` verwenden!
 
**Kein Bezug zur Modellunsicherheit oder Schwellenwerten (threshold):**
 
Model sagt: Mit 0,91 Wahrscheinlichkeit ist das Beispiel Klasse 1 z.B. krank
 
```python
model.predict_proba(x) # [0,09, 0,91]
```
 
| Modell | Wahrscheinlichkeit für "krank" | Schwellenwert | Vorhersage | Ergebnis |
| ------ | ------------------------------ | ------------- | ---------- | -------- |
| A      | 0.99                           | 0.5           | krank      | richtig  |
| B      | 0.51                           | 0.5           | krank      | richtig  |
 

## 2. ROC-AUC (Receiver Operating Characteristic - Area Under Curve)

- Wie gut kann ein Klassifikationsmodell zwischen positiven und negativen Klassen unterscheiden - unabhängig davon, welche Schwellenwerte man zur Klassifikation verwendet?

- Auf x-achse Falsch Positive Rate (FPR)
- Auf y-achse True Positive Rate (TPR)

In [9]:
y_probs = clf.predict_proba(X_test)

In [10]:
y_probs_positive = y_probs[:, 1]
y_probs_positive

array([0.08, 0.58, 0.51, 0.09, 0.76, 0.83, 0.67, 0.03, 0.05, 0.6 , 0.77,
       0.22, 0.92, 0.1 , 0.95, 1.  , 1.  , 0.13, 0.02, 0.08, 0.59, 0.07,
       0.64, 0.76, 0.77, 0.67, 0.8 , 0.71, 0.1 , 0.8 , 0.08, 0.07, 0.04,
       0.31, 0.58, 0.08, 0.63, 0.82, 0.64, 0.85, 0.83, 0.79, 0.84, 0.71,
       0.72, 0.41, 0.61, 0.97, 0.12, 0.06, 0.3 , 0.25, 0.75, 0.71, 0.17,
       0.12, 0.34, 0.95, 0.12, 0.02, 0.16])

In [11]:
from sklearn.metrics import roc_curve, auc
import plotly.graph_objects as go

# ROC-Werte berechnen:
fpr, tpr, thresholds = roc_curve(y_test, y_probs_positive)

# AUC-Werte berechnen:
roc_auc = auc(fpr, tpr)

# ROC-Kurve zeichnen:

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=fpr,
    y=tpr,
    mode="lines",
    name="ROC-Kurve",
    line=dict(color="orange"),
    hovertemplate="FPR: %{x:.2f}<br>TPR: %{y:.2f}"
))

# Diagonale (Zufallsmodell) zeichnen:
fig.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode="lines",
    name="Zufallsmodel (AUC = 0.5)",
    line=dict(color="gray", dash="dash")
))

fig.update_layout(
    title=f"ROC-Kurve - Modellgüte über alle Schwellenwerte (AUC = {roc_auc:.3f})",
    xaxis_title="Falsch Positive Rate (TPR / Sensitivität)",
    template="plotly_white",
    width=800,
    height=500
)

fig

In [12]:
import numpy as np

# Abstand zu Punkt (0, 1) berechnen:
distances = np.sqrt((fpr - 0)**2 + (tpr - 1)**2)

# Index des kleinsten Abstand finden:
optimal_idx = np.argmin(distances)

optimal_threshold = thresholds[optimal_idx]

print(f"{optimal_threshold:.2f}")



0.51


In [13]:
fpr[optimal_idx]

np.float64(0.1724137931034483)

In [14]:
tpr[optimal_idx]



np.float64(0.90625)

In [15]:
y_pred =(y_probs_positive >= optimal_threshold).astype(int)
accuracy_score(y_test, y_pred)

0.8688524590163934

## 3. Confusion-Matrix

In [16]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

cm_df = pd.DataFrame(
    cm,
    index=["Tatsächlich: Gesund(0)", "Tatsächlich: Krank(1)"],
    columns=["Vorhergesagt: Gesund (0)", "Vorhergesagt: Krank (1)"]
)

cm_df

[[24  5]
 [ 3 29]]


,Vorhergesagt: Gesund (0),Vorhergesagt: Krank (1)
Tatsächlich: Gesund(0),24,5
Tatsächlich: Krank(1),3,29


In [17]:
cell_colors = [
    ["#92D695", "#F5CBCB"],
    ["#FFDE63", "#6a51a3"]
]

x_labels = ["Vorhergesagt: Gesund", "Vorhergesagt: Krank"]
y_labels = ["Tatsächlich: Krank", "Tatsächlich: Gesund"]

legend_items = [
    ("True Negative (TN)", "#92D695"),
    ("False Positive (FP)", "#F5CBCB"),
    ("False Negative (FN)", "#FFDE63"),
    ("True Positive (TP)","#6a51a3")
]

fig = go.Figure()

# Rechteecke und Beschriftungen hinzufügen:
for i in range(2): # Zeilen
    for j in range(2): # Spalten
        fig.add_shape(
            type="rect",
            x0=j, x1=j + 1,
            y0=i, y1=i + 1,
            fillcolor=cell_colors[i][j],
            line=dict(color="black")
        )
        
        # Zahlen zentriert hinzufügen:
        fig.add_annotation(
            x=j + 0.5,
            y=i + 0.5,
            text=str(cm[i][j]),
            showarrow=False,
            font=dict(size=20)
        )

# Legende:
for label, color in legend_items:
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode="markers",
        showlegend=True,
        marker=dict(size=10, color=color),
        legendgroup=label,
        name=label
    ))
    

fig.update_layout(
    width=900,
    height=600,
    xaxis=dict(
        tickvals=[0.5, 1.5],
        ticktext=x_labels,
        range=[0, 2],
        showgrid=False
    ),
    yaxis=dict(
        tickvals=[0.5, 1.5],
        ticktext=y_labels[::-1],
        range=[0, 2],
        showgrid=False
    ),
    title="Confusion-Matrix",
    plot_bgcolor="white"
)
        
fig

## 4. Der Classification Report

- Precision
- Recall
- F1-Score
- Support

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=["Gesund", "Herzkrank"]))

              precision    recall  f1-score   support

      Gesund       0.89      0.83      0.86        29
   Herzkrank       0.85      0.91      0.88        32

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.87      0.87      0.87        61

